In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from sklearn.decomposition import PCA
from pyntcloud import PyntCloud

In [2]:
pcd = o3d.io.read_point_cloud('C:/Users/operator/Desktop/point_cloud_gt/clouds_stereo/cloud_2_0013.pcd', remove_nan_points  = True, remove_infinite_points = True)

In [3]:
pcd

PointCloud with 741245 points.

In [4]:
#o3d.visualization.draw_geometries([pcd])

In [5]:
point_cloud = np.asarray(pcd.points)

In [6]:
point_cloud

array([[-1.773    , -1.08     ,  1.4150001],
       [-1.77     , -1.08     ,  1.4170001],
       [-1.767    , -1.08     ,  1.4250001],
       ...,
       [ 1.899    ,  1.077    ,  3.2490001],
       [ 1.9020001,  1.077    ,  3.2390001],
       [ 1.905    ,  1.077    ,  3.2290001]])

In [7]:
'''
def grid_subsampling(points, voxel_size):
    nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(((points - np.min(points, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted=np.argsort(inverse)
    voxel_grid={}
    grid_barycenter,grid_candidate_center=[],[]
    last_seen=0

    for idx,vox in enumerate(non_empty_voxel_keys):
        voxel_grid[tuple(vox)]=points[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]]
        grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))
        grid_candidate_center.append(voxel_grid[tuple(vox)][np.linalg.norm(voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()])
        last_seen+=nb_pts_per_voxel[idx]

    return grid_candidate_center
grid_sampled_point_cloud_variable = grid_subsampling(point_cloud, 0.05)
pcd_0 = o3d.geometry.PointCloud()
pcd_0.points = o3d.utility.Vector3dVector(grid_sampled_point_cloud_variable)
o3d.visualization.draw_geometries([pcd_0])
'''

'\ndef grid_subsampling(points, voxel_size):\n    nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)\n    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(((points - np.min(points, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)\n    idx_pts_vox_sorted=np.argsort(inverse)\n    voxel_grid={}\n    grid_barycenter,grid_candidate_center=[],[]\n    last_seen=0\n\n    for idx,vox in enumerate(non_empty_voxel_keys):\n        voxel_grid[tuple(vox)]=points[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]]\n        grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))\n        grid_candidate_center.append(voxel_grid[tuple(vox)][np.linalg.norm(voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()])\n        last_seen+=nb_pts_per_voxel[idx]\n\n    return grid_candidate_center\ngrid_sampled_point_cloud_variable = grid_subsampling(point_cloud, 0.05)\npcd_0 = o3d.geometry.PointClou

In [8]:
'''
tree = KDTree(np.asarray(pcd_0.points), leafsize = 100)
point = point_cloud[0]  
# Recover the k closest points of out reference point
dist_to_neighbors, neighbor_indices = tree.query(point, 10000)
neighborhoods = point_cloud[neighbor_indices]
pcd_neighborhoods_0 = o3d.geometry.PointCloud()
pcd_neighborhoods_0.points = o3d.utility.Vector3dVector(neighborhoods)
o3d.visualization.draw_geometries([pcd_neighborhoods_0])
'''

'\ntree = KDTree(np.asarray(pcd_0.points), leafsize = 100)\npoint = point_cloud[0]  \n# Recover the k closest points of out reference point\ndist_to_neighbors, neighbor_indices = tree.query(point, 10000)\nneighborhoods = point_cloud[neighbor_indices]\npcd_neighborhoods_0 = o3d.geometry.PointCloud()\npcd_neighborhoods_0.points = o3d.utility.Vector3dVector(neighborhoods)\no3d.visualization.draw_geometries([pcd_neighborhoods_0])\n'

In [9]:
'''
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02)
print("Statistical oulier removal")
cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=10, radius=0.05)
display_inlier_outlier(voxel_down_pcd, ind)
'''

'\ndef display_inlier_outlier(cloud, ind):\n    inlier_cloud = cloud.select_by_index(ind)\n    outlier_cloud = cloud.select_by_index(ind, invert=True)\n\n    print("Showing outliers (red) and inliers (gray): ")\n    outlier_cloud.paint_uniform_color([1, 0, 0])\n    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])\n    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])\n\nvoxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.02)\nprint("Statistical oulier removal")\ncl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=10, radius=0.05)\ndisplay_inlier_outlier(voxel_down_pcd, ind)\n'

In [10]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6, 0.6, 0.6])

PointCloud with 741245 points.

In [11]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.2, ransac_n=2000, num_iterations=5000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

Plane equation: 0.00x + 0.00y + 1.00z + -3.28 = 0


PointCloud with 169586 points.

In [12]:
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10))
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
#o3d.visualization.draw_geometries([pcd])

In [13]:
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [14]:
#o3d.visualization.draw_geometries([outlier_cloud])

In [15]:
plane_model_1, inliers_1 = outlier_cloud.segment_plane(distance_threshold=0.2, ransac_n=2000, num_iterations=5000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud_1 = outlier_cloud.select_by_index(inliers_1)
outlier_cloud_1 = outlier_cloud.select_by_index(inliers_1, invert=True)
inlier_cloud_1.paint_uniform_color([1.0, 0, 0])
outlier_cloud_1.paint_uniform_color([0.6, 0.6, 0.6])

Plane equation: 0.00x + 0.00y + 1.00z + -3.28 = 0


PointCloud with 63861 points.

In [16]:
#o3d.visualization.draw_geometries([inlier_cloud_1, outlier_cloud_1])

In [17]:
#o3d.visualization.draw_geometries([outlier_cloud_1])

In [18]:
labels = np.array(outlier_cloud_1.cluster_dbscan(eps=0.05, min_points=20))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud_1.colors = o3d.utility.Vector3dVector(colors[:, :3])

#o3d.visualization.draw_geometries([outlier_cloud_1])

point cloud has 77 clusters


In [19]:
'''
data_mean = np.mean(point_cloud, axis=0) #Calculate the average value of the column
# Normalized 
normalize_data = point_cloud - data_mean
pcd_norma = o3d.geometry.PointCloud()
pcd_norma.points = o3d.utility.Vector3dVector(normalize_data)
o3d.visualization.draw_geometries([pcd_norma])
'''

'\ndata_mean = np.mean(point_cloud, axis=0) #Calculate the average value of the column\n# Normalized \nnormalize_data = point_cloud - data_mean\npcd_norma = o3d.geometry.PointCloud()\npcd_norma.points = o3d.utility.Vector3dVector(normalize_data)\no3d.visualization.draw_geometries([pcd_norma])\n'